In [1]:
# Import necessary libraries to run AutoML
import sys, os, os.path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle

import h2o
from h2o.automl import H2OAutoML

from sklearn.model_selection import train_test_split

## Initalize functions and constants

In [ ]:
locations = ["A", "B", "C"]
scaler = None
features_order = []

LAGGED_COLUMNS_TO_KEEP = [
    'direct_rad:W_lag_1h', 
    'direct_rad:W_lag_forward_1h', 
    'clear_sky_rad:W_lag_1h', 
    'clear_sky_rad:W_lag_forward_1h', 
    'diffuse_rad:W_lag_1h', 
    'diffuse_rad:W_lag_forward_1h', 
    'direct_rad_1h:J_lag_1h', 
    'direct_rad_1h:J_lag_forward_1h', 
    'is_in_shadow:idx_lag_1h', 
    'is_in_shadow:idx_lag_forward_1h', 
    'clear_sky_energy_1h:J_lag_1h', 
    'clear_sky_energy_1h:J_lag_forward_1h', 
    'effective_cloud_cover:p_lag_1h', 
    'effective_cloud_cover:p_lag_forward_1h', 
    'visibility:m_lag_1h', 
    'visibility:m_lag_forward_1h', 
    'total_cloud_cover:p_lag_1h', 
    'total_cloud_cover:p_lag_forward_1h', 

    'direct_rad:W_lag_3h', 
    'direct_rad:W_lag_forward_3h', 
    'clear_sky_rad:W_lag_3h', 
    'clear_sky_rad:W_lag_forward_3h', 
    'diffuse_rad:W_lag_3h', 
    'diffuse_rad:W_lag_forward_3h', 
    'direct_rad_1h:J_lag_3h', 
    'direct_rad_1h:J_lag_forward_3h', 
    'is_in_shadow:idx_lag_3h', 
    'is_in_shadow:idx_lag_forward_3h', 
    'clear_sky_energy_1h:J_lag_3h', 
    'clear_sky_energy_1h:J_lag_forward_3h', 
    'effective_cloud_cover:p_lag_3h', 
    'effective_cloud_cover:p_lag_forward_3h', 
    'visibility:m_lag_3h', 
    'visibility:m_lag_forward_3h', 
    'total_cloud_cover:p_lag_3h', 
    'total_cloud_cover:p_lag_forward_3h'
]

CUSTOM_COLUMNS_TO_KEEP = [
    "hour_cos",
    "hour_sin",
    "month_sin",
    "month_cos",
    "day-of-year",
]

WEATHER_FEATURES = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "effective_cloud_cover:p",
    "visibility:m",
    "total_cloud_cover:p",
]


COLUMNS_TO_KEEP = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "diffuse_rad_1h:J",
    "is_day:idx",
    "sun_elevation:d",
    # "ceiling_height_agl:m",
    "effective_cloud_cover:p",
    "visibility:m",
    "total_cloud_cover:p",
    "air_density_2m:kgm3",
    "wind_speed_v_10m:ms",
    "dew_point_2m:K",
    "wind_speed_u_10m:ms",
    "t_1000hPa:K",
    "absolute_humidity_2m:gm3",
     "snow_water:kgm2",
    "relative_humidity_1000hPa:p",
    "fresh_snow_24h:cm",
    "cloud_base_agl:m",
    "fresh_snow_12h:cm",
    "snow_depth:cm",
    "dew_or_rime:idx",
    "fresh_snow_6h:cm",
    "super_cooled_liquid_water:kgm2",
    "fresh_snow_3h:cm",
    "rain_water:kgm2",
    "precip_type_5min:idx",
    "precip_5min:mm",
    "fresh_snow_1h:cm",
    "sun_azimuth:d",
    "msl_pressure:hPa",
    "pressure_100m:hPa",
    "pressure_50m:hPa",
    "sfc_pressure:hPa",
    "prob_rime:p",
    "wind_speed_10m:ms",
    "elevation:m",
    # "snow_density:kgm3",
    "snow_drift:idx",
    "snow_melt_10min:mm",
    "wind_speed_w_1000hPa:ms",
    # "date_calc",
    "pv_measurement",
] + CUSTOM_COLUMNS_TO_KEEP + LAGGED_COLUMNS_TO_KEEP

TEST_COLUMNS_TO_KEEP = [
    "direct_rad:W",
    "clear_sky_rad:W",
    "diffuse_rad:W",
    "direct_rad_1h:J",
    "is_in_shadow:idx",
    "clear_sky_energy_1h:J",
    "diffuse_rad_1h:J",
    "is_day:idx",
    "sun_elevation:d",
    # "ceiling_height_agl:m",
    "effective_cloud_cover:p",
    "visibility:m",
    "total_cloud_cover:p",
    "air_density_2m:kgm3",
    "wind_speed_v_10m:ms",
    "dew_point_2m:K",
    "wind_speed_u_10m:ms",
    "t_1000hPa:K",
    "absolute_humidity_2m:gm3",
    "snow_water:kgm2",
    "relative_humidity_1000hPa:p",
    "fresh_snow_24h:cm",
    "cloud_base_agl:m",
    "fresh_snow_12h:cm",
    "snow_depth:cm",
    "dew_or_rime:idx",
    "fresh_snow_6h:cm",
    "super_cooled_liquid_water:kgm2",
    "fresh_snow_3h:cm",
    "rain_water:kgm2",
    "precip_type_5min:idx",
    "precip_5min:mm",
    "fresh_snow_1h:cm",
    "sun_azimuth:d",
    "msl_pressure:hPa",
    "pressure_100m:hPa",
    "pressure_50m:hPa",
    "sfc_pressure:hPa",
    "prob_rime:p",
    "wind_speed_10m:ms",
    "elevation:m",
    # "snow_density:kgm3",
    "snow_drift:idx",
    "snow_melt_10min:mm",
    "wind_speed_w_1000hPa:ms",
    # "date_calc",
    # "pv_measurement",
] + CUSTOM_COLUMNS_TO_KEEP + LAGGED_COLUMNS_TO_KEEP



def create_weather_lagged_features(df, weather_features):
    # Choose the weather features for which you want to create lagged versions
    for feature in weather_features:
        # Assuming hourly data, adjust the lags for your specific dataset
        # Creating lagged features for 1 hour, 1 day, and 1 week
        df[f'{feature}_lag_1h'] = df[feature].shift(1)
        df[f'{feature}_lag_3h'] = df[feature].shift(2)
        df[f'{feature}_lag_forward_1h'] = df[feature].shift(-1)
        df[f'{feature}_lag_forward_3h'] = df[feature].shift(-2)
        # df[f'{feature}_lag_24h'] = df[feature].shift(24*4)
        # df[f'{feature}_lag_168h'] = df[feature].shift(24 * 7 * 4 * 365)
        # df[f'{feature}_front_lag_1h'] = df[feature].shift(-4)
        # df[f'{feature}_front_lag_24h'] = df[feature].shift(-24*4)


    # Handling edges by filling NaNs with appropriate values or dropping them
    # You may choose to fill with zeroes or interpolate, based on what makes more sense for your data
    df.fillna(method='ffill', inplace=True)  # Forward fill
    df.fillna(method='bfill', inplace=True)  # Backward fill
    
    return df


def create_lagged_features(df, column_name='pv_measurement'):
    # Assuming 'date_forecast' is the datetime column used for sorting

    df[f'{column_name}_prev_month'] = df[column_name].shift(24*7) # previous week

    # For yearly lag, you would need to calculate the number of observations per year
    # If the data is not consistent (leap years, etc.), you may need a more complex method
    # Here's a simple version assuming 365 days a year:
    df[f'{column_name}_prev_year'] = df[column_name].shift(24*365) # previous year
    df[f'{column_name}_2years_ago'] = df[column_name].shift(24*365*2) # next year

    # Handling edges by filling NaNs with appropriate values or dropping them
    df.fillna(method='ffill', inplace=True)  # Forward fill
    df.fillna(method='bfill', inplace=True)  # Backward fill

    return df


## Prepare data

In [ ]:
def add_custom_fields(df):
     df['hour_sin'] = np.sin(2 * np.pi * df['date_forecast'].dt.hour / 24)
     df['hour_cos'] = np.cos(2 * np.pi * df['date_forecast'].dt.hour / 24)

     df['month_sin'] = np.sin(2 * np.pi * df['date_forecast'].dt.month / 12)
     df['month_cos'] = np.cos(2 * np.pi * df['date_forecast'].dt.month / 12)
     df['day-of-year'] = df['date_forecast'].dt.dayofyear
     return df

def remove_outliers(df):
    # Use a mask to filter out the rows where rolling std is zero but keep the rows where the value itself is zero
    mask = (df['pv_measurement'].rolling(5).std() == 0) & (df['pv_measurement'] != 0)
    df = df[~mask]
    return df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize a dictionary to hold the scalers for each location
scalers = {}

def prepare_data(location):
    # Load data
    scaling = False  # Set scaling to True to enable individual scaling for each location
    global features_order
    global scalers  # Use the global dictionary to store scalers

    df_observed = pd.read_parquet(f"data/{location}/X_train_observed.parquet")
    df_estimated = pd.read_parquet(f"data/{location}/X_train_estimated.parquet")
    df_target = pd.read_parquet(f"data/{location}/train_targets.parquet")

    # Combine observed and estimated datasets
    df_combined = pd.concat([df_observed, df_estimated], axis=0).sort_values(by="date_forecast")

    df_combined = add_custom_fields(df_combined)
    df_combined.set_index('date_forecast', inplace=True)
    df_merged = df_combined.resample('1H').mean()

    # Merge with target data
    df_merged = pd.merge(df_merged, df_target, left_on="date_forecast", right_on="time", how="inner")
    
    # One-hot encoding for location
    df_merged['location'] = location
    df_merged['location_A'] = (df_merged['location'] == 'A').astype(int)
    df_merged['location_B'] = (df_merged['location'] == 'B').astype(int)
    df_merged['location_C'] = (df_merged['location'] == 'C').astype(int)
    df_merged.drop('location', axis=1, inplace=True)

    df_merged = create_weather_lagged_features(df_merged, WEATHER_FEATURES)
    df_merged = df_merged[COLUMNS_TO_KEEP]
    df_merged = remove_outliers(df_merged)

    y = df_merged["pv_measurement"]
    X = df_merged.drop("pv_measurement", axis=1)
    features_order = list(X.columns)
    df_merged.fillna(0, inplace=True)  # Fill NaN values

    # Apply Min-Max normalization to the features
    if scaling: 
        # Check if a scaler for this location already exists
        if location not in scalers:
            scalers[location] = MinMaxScaler()
            X_normalized = scalers[location].fit_transform(X)
        else:
            X_normalized = scalers[location].transform(X)

        return pd.DataFrame(X_normalized, columns=X.columns), y
    else: 
        return X, y


## Training 

In [ ]:
# Initialize H2O cluster
h2o.init(
    nthreads=-1,     # number of threads when launching a new H2O server
    max_mem_size=8  # in gigabytes
)
X.columns


: 

: 

In [ ]:
# Columns to train on

cols = ['absolute_humidity_2m:gm3', 'air_density_2m:kgm3',
       'ceiling_height_agl:m', 'clear_sky_energy_1h:J', 'clear_sky_rad:W',
       'cloud_base_agl:m', 'dew_point_2m:K', 'diffuse_rad:W',
       'diffuse_rad_1h:J', 'direct_rad:W', 'direct_rad_1h:J',
       'effective_cloud_cover:p',
       'is_day:idx', 'is_in_shadow:idx', 'msl_pressure:hPa', 'precip_5min:mm',
       'precip_type_5min:idx', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'prob_rime:p', 'rain_water:kgm2', 'relative_humidity_1000hPa:p',
       'sfc_pressure:hPa', 
       'sun_azimuth:d', 'sun_elevation:d',
       't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'wind_speed_w_1000hPa:ms', 'location', 'direct_rad:W_ls',
       'effective_cloud_cover:p_ls', 'clear_sky_rad:W_ls',
       'sun_elevation:d_ls', 'diffuse_rad:W_ls', 'dew:idx', 'rime:idx',
       'hour_diff', 'pv_measurement']


: 

In [ ]:
# Convert data into H2OFrame
train = h2o.H2OFrame(pd.concat([X_train, y_train], axis=1))
test = h2o.H2OFrame(pd.concat([X_test, y_test], axis=1))

x = cols
y = "pv_measurement"
x.remove(y)

# Set up max runtime and seed for AutoML
aml = H2OAutoML(max_runtime_secs=3600, seed=42)

: 

In [ ]:
# Train AutoML model
aml.train(x=x, y=y, training_frame=train)

: 

In [ ]:
# List of columns used during training
columns_used_in_training = train.columns
columns_used_in_training

: 

In [ ]:
# View the AutoML Leaderboard
lb = aml.leaderboard
model_ids = list(lb['model_id'].as_data_frame().iloc[:,0])
lb.head()

: 

In [ ]:
# Get the best model
best_model = aml.get_best_model()
print(best_model)

: 

In [ ]:
# Save the three best models
leaderboard = aml.leaderboard

top5_model_ids = h2o.as_list(leaderboard['model_id'])['model_id'][:5]

for idx, model_id in enumerate(top5_model_ids, 1):
    model = h2o.get_model(model_id)
    h2o.save_model(model=model, path=f"../data_folder/models/model_ls_{idx}", force=True)
    
   


: 

In [ ]:
# Inspect the model's performance on the test data
perf = best_model.model_performance(test)
print(perf)


: 

In [ ]:
# Show the model's actual predictions and the true values in the same dataframe
pred = best_model.predict(test)
pred_df = pred.as_data_frame()
test_df = test.as_data_frame()
test_df.reset_index(drop=True, inplace=True)
pred_df.reset_index(drop=True, inplace=True)
merged_df = pd.concat([test_df, pred_df], axis=1)
merged_df.rename(columns={'predict': 'prediction'}, inplace=True)
merged_df.head()




: 

In [ ]:
explain_model = aml.explain(frame = train, figsize = (8,6))

: 

: 